In [1]:
import matplotlib.pyplot as plt
import torch
import norse
import numpy as np

In [2]:
# Set up the leaky integrate-and-fire (LI) neuron model using the Norse library
activation = norse.torch.LI()
activation

LI()

In [3]:
# Generate input data with a sequence of 1000 time steps and specific spikes
data = torch.zeros(1000, 1)
data[20] = 1.0
data[100] = 1.0

# Run the LI neuron model with the input data and capture the resulting voltage over time
voltage_trace, _ = activation(data)

In [ ]:
# Define fixed values for tau_mem and tau_syn
tau_mem = 105  # Middle value of 10 to 200 range
tau_syn = 105  # Middle value of 10 to 200 range
t0 = 500  # Middle value of 0 to 1000 range
t1 = 700  # Middle value of 0 to 1000 range

num_neurons = 1
tau_mem_inv = torch.tensor([1/(tau_mem * 0.001)])
tau_syn_inv = torch.tensor([1/(tau_syn * 0.001)])

# Create a time-series input with spikes at specified times
data = torch.zeros(1000, num_neurons)
data[t0] = 1.0
data[t1] = 1.0

# Simulate the neuron model with fixed parameters and capture the voltage trace
voltage_trace, _ = norse.torch.LI(p=norse.torch.LIParameters(tau_mem_inv=tau_mem_inv, tau_syn_inv=tau_syn_inv))(data)


Increasing the membrane time constant (τ_mem) slows down the decay of the membrane potential, resulting in a lower peak value. The synaptic time constant (τ_syn) affects the exponential decay rate of the synaptic input.

In [ ]:
# Function to simulate and record the voltage of a neuron model
def integrate_and_record_voltages(cell):
    def integrate(input_spike_train):
        T = input_spike_train.shape[0]
        s = None
        spikes = []
        voltage_trace = []

        # Process each time step to simulate neuron activity and capture spikes and voltages
        for ts in range(T):
            z, s = cell(input_spike_train[ts], s)
            spikes.append(z)
            voltage_trace.append(s.v)

        return torch.stack(spikes), torch.stack(voltage_trace)
    return integrate

In [ ]:
# Define fixed values for tau_mem, tau_syn, and v_th
tau_mem = 105  # Middle value of 10 to 200 range
tau_syn = 105  # Middle value of 10 to 200 range
v_th = 1.05    # Middle value of 0.1 to 2.0 range
t0 = 500       # Middle value of 0 to 1000 range
t1 = 700       # Middle value of 0 to 1000 range

num_neurons = 1
tau_syn_inv = torch.tensor([1/(tau_syn * 0.001)])
tau_mem_inv = torch.tensor([1/(tau_mem * 0.001)])

# Create a signal with spikes at specified time points
data = torch.zeros(1000, num_neurons)
data[20] = 1.0
data[t0] = 1.0
data[t1] = 1.0

# Configure the LIF neuron model with fixed parameters
cell = norse.torch.LIFCell(p=norse.torch.LIFParameters(tau_mem_inv=tau_mem_inv, tau_syn_inv=tau_syn_inv, v_th=torch.as_tensor(v_th)))
lif_integrate = integrate_and_record_voltages(cell)

# Simulate the LI and LIF models and capture their voltage traces
voltage_trace, _ = norse.torch.LI(p=norse.torch.LIParameters(tau_mem_inv=tau_mem_inv, tau_syn_inv=tau_syn_inv))(data)
zs, lif_voltage_trace = lif_integrate(data)